In [7]:
! pip uninstall -y tensorflow

Found existing installation: tensorflow 2.9.1
Uninstalling tensorflow-2.9.1:
  Successfully uninstalled tensorflow-2.9.1


In [8]:
! pip install tensorflow

  Using cached tensorflow-2.9.1-cp39-cp39-macosx_10_14_x86_64.whl (228.5 MB)


In [1]:
%cd /Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/

/Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt


In [13]:
import tensorflow as tf

In [3]:
# GLobal definations

path = '/Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/'
block_h, block_w = (32, 32)
Sub_Model = 5

In [4]:
# Function Definition --> Loading preprocessed data from the file & Normalize it.

def Load_Data():
    pickle_in = open(Case_Path + '/' + 'X_train.pickle', 'rb')
    X = pickle.load(pickle_in)  # Loading Features

    pickle_in = open(Case_Path + '/' + 'y_train.pickle', 'rb')
    y = pickle.load(pickle_in)  # Loading Labels

    print('Shape of X:\t', X.shape), print('Shape of y:\t', y.shape)

    return X, y

In [22]:
# Function Definition --> For Development, Compilation, Training & Storing of the Model

def Model(img_h, img_w, X_train, y_train, X_test, y_test, count):
    # import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, LeakyReLU, Activation
    from tensorflow.keras.utils import to_categorical

    model = Sequential()

    model.add (Conv2D (32, (3, 3), padding = 'same', input_shape = (img_h, img_w, 1)))
    model.add (LeakyReLU (alpha = 0.1))
    model.add (MaxPooling2D (pool_size = (3, 3)))  # Pool_size is the size of filter.
    model.add (Dropout(0.1))

    model.add (Conv2D (48, (3, 3), padding = 'same'))
    model.add (LeakyReLU (alpha = 0.1))
    model.add (MaxPooling2D (pool_size = (3, 3)))
    model.add (Dropout(0.1))

    model.add (Conv2D (48, (3, 3), padding = 'same'))
    model.add (LeakyReLU (alpha = 0.1))
    model.add (MaxPooling2D (pool_size = (3, 3)))
    model.add (Dropout(0.1))

    model.add (Flatten())  # Convert 3D feature map to 1D feature vector.

    model.add (Dense(1096))
    model.add (LeakyReLU (alpha = 0.1))
    model.add (Dropout(0.1))
    model.add (Dense(2, activation = 'softmax'))

    # Compiling the model
    model.compile (loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.optimizer.learning_rate = 0.0001

    # Training the model & storing its detail in the variable h
    history = model.fit (X_train, to_categorical(y_train), batch_size = 128, epochs = 50, verbose = 2,
               validation_data=(X_test, to_categorical(y_test)), shuffle = True)

    # Evaluating performance of the Model
    test_loss, test_acc = model.evaluate (X_test, to_categorical(y_test))
    results.append(test_acc)
    print('Test Loss is: ', test_loss, '\nTest Accuracy is: ', test_acc)

    # Saving the model in hierarchical Data Formate (HDF)
    model.save (path + 'Outputs/Model_' + case + '_' + str(count+1) + '.h5')

    return history, model

In [6]:
import tensorflow


In [8]:
# Function Definition --> For Saving Validation Accuracy in the file

def Saving_Test_Accuracy():
    import numpy as np
    
    file = open(Result_File, "a")
    file.write(case)
    file.write(':\n======')
    file.write('\nAll Test Accuracy:\t')
    
    for ans in results:
        file.write(str(ans) + ', ')
    
    file.write('\nMean Test Accuracy:\t' + str(np.mean(results)))
    file.write('\nMin Test Accuracy:\t' + str(np.min(results)))
    file.write('\nMax Test Accuracy:\t' + str(np.max(results)))
    file.write('\n\n\n')
    file.close()

In [23]:
# All Function Calls
import os
from tqdm import tqdm
import pickle
from sklearn.model_selection import train_test_split
from random import randint

PATH = path + 'Data/'
Work_Dir = PATH + 'Working_Data/'
Result_File = path + 'Outputs/Results.txt'

# Calling Functions:
osdir=os.listdir(Work_Dir)
if '.DS_Store' in osdir:
    osdir.remove('.DS_Store')

for case in tqdm(osdir):
    Case_Path = os.path.join(Work_Dir, case)  # Joining path for "Cases'.
    print(case)

    # Function Call --> Loading preprocessed data from the file & Normalize it.
    X, y = Load_Data()

    results = []
    for i in range(Sub_Model):
        print('\n' + case + '_' + str(i+1))
        
        # Spliting data into two parts, "Training = 70%" and "Testing = 30%".
        num = randint(0,1000)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = num)

        # Function Call --> For Development, Compilation, Training & Storing of the Model
        history, model = Model(block_h, block_w, X_train, y_train, X_test, y_test, i)
    print('\n')

    # Function Call --> For Saving Validation Accuracy in the file
    Saving_Test_Accuracy()
print('Done...!!')

  0%|          | 0/5 [00:00<?, ?it/s]2022-09-02 02:19:03.804750: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-02 02:19:03.805342: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Case_5
Shape of X:	 (32148, 32, 32, 1)
Shape of y:	 (32148,)

Case_5_1
Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Epoch 1/50


2022-09-02 02:19:04.637616: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-09-02 02:19:04.991899: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-02 02:19:11.676097: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


176/176 - 8s - loss: 1.0268 - accuracy: 0.5172 - val_loss: 0.6870 - val_accuracy: 0.5705 - 8s/epoch - 43ms/step
Epoch 2/50
176/176 - 3s - loss: 0.7111 - accuracy: 0.5486 - val_loss: 0.6782 - val_accuracy: 0.5844 - 3s/epoch - 15ms/step
Epoch 3/50
176/176 - 3s - loss: 0.6954 - accuracy: 0.5613 - val_loss: 0.6721 - val_accuracy: 0.5838 - 3s/epoch - 15ms/step
Epoch 4/50
176/176 - 3s - loss: 0.6807 - accuracy: 0.5741 - val_loss: 0.6932 - val_accuracy: 0.5492 - 3s/epoch - 15ms/step
Epoch 5/50
176/176 - 3s - loss: 0.6752 - accuracy: 0.5866 - val_loss: 0.7272 - val_accuracy: 0.5532 - 3s/epoch - 15ms/step
Epoch 6/50
176/176 - 3s - loss: 0.6707 - accuracy: 0.5909 - val_loss: 0.7314 - val_accuracy: 0.5564 - 3s/epoch - 15ms/step
Epoch 7/50
176/176 - 3s - loss: 0.6628 - accuracy: 0.6031 - val_loss: 0.6976 - val_accuracy: 0.5582 - 3s/epoch - 15ms/step
Epoch 8/50
176/176 - 3s - loss: 0.6527 - accuracy: 0.6187 - val_loss: 0.8268 - val_accuracy: 0.5550 - 3s/epoch - 15ms/step
Epoch 9/50
176/176 - 3s - l

2022-09-02 02:21:25.865613: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-02 02:21:28.520037: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

176/176 - 4s - loss: 0.9840 - accuracy: 0.5234 - val_loss: 0.7114 - val_accuracy: 0.4727 - 4s/epoch - 24ms/step
Epoch 2/50
176/176 - 3s - loss: 0.7103 - accuracy: 0.5405 - val_loss: 0.7107 - val_accuracy: 0.4832 - 3s/epoch - 18ms/step
Epoch 3/50
176/176 - 3s - loss: 0.6953 - accuracy: 0.5579 - val_loss: 0.6653 - val_accuracy: 0.5841 - 3s/epoch - 18ms/step
Epoch 4/50
176/176 - 3s - loss: 0.6814 - accuracy: 0.5770 - val_loss: 0.6548 - val_accuracy: 0.6115 - 3s/epoch - 17ms/step
Epoch 5/50
176/176 - 3s - loss: 0.6695 - accuracy: 0.5959 - val_loss: 0.6519 - val_accuracy: 0.6582 - 3s/epoch - 18ms/step
Epoch 6/50
176/176 - 3s - loss: 0.6600 - accuracy: 0.6095 - val_loss: 0.6380 - val_accuracy: 0.6225 - 3s/epoch - 18ms/step
Epoch 7/50
176/176 - 3s - loss: 0.6520 - accuracy: 0.6199 - val_loss: 0.6381 - val_accuracy: 0.5952 - 3s/epoch - 18ms/step
Epoch 8/50
176/176 - 3s - loss: 0.6401 - accuracy: 0.6342 - val_loss: 0.6448 - val_accuracy: 0.5842 - 3s/epoch - 18ms/step
Epoch 9/50
176/176 - 3s - l

2022-09-02 03:10:03.494246: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-02 03:10:06.508403: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


176/176 - 4s - loss: 1.0372 - accuracy: 0.5185 - val_loss: 0.6876 - val_accuracy: 0.5628 - 4s/epoch - 22ms/step
Epoch 2/50
176/176 - 3s - loss: 0.7089 - accuracy: 0.5437 - val_loss: 0.6758 - val_accuracy: 0.6166 - 3s/epoch - 18ms/step
Epoch 3/50
176/176 - 3s - loss: 0.6901 - accuracy: 0.5661 - val_loss: 0.6738 - val_accuracy: 0.6032 - 3s/epoch - 18ms/step
Epoch 4/50
176/176 - 3s - loss: 0.6813 - accuracy: 0.5784 - val_loss: 0.6644 - val_accuracy: 0.6377 - 3s/epoch - 17ms/step
Epoch 5/50
176/176 - 3s - loss: 0.6738 - accuracy: 0.5910 - val_loss: 0.6546 - val_accuracy: 0.6105 - 3s/epoch - 17ms/step
Epoch 6/50
176/176 - 3s - loss: 0.6660 - accuracy: 0.6049 - val_loss: 0.6458 - val_accuracy: 0.6055 - 3s/epoch - 18ms/step
Epoch 7/50
176/176 - 3s - loss: 0.6574 - accuracy: 0.6126 - val_loss: 0.6623 - val_accuracy: 0.5600 - 3s/epoch - 18ms/step
Epoch 8/50
176/176 - 3s - loss: 0.6463 - accuracy: 0.6294 - val_loss: 0.6607 - val_accuracy: 0.5652 - 3s/epoch - 18ms/step
Epoch 9/50
176/176 - 3s - l

2022-09-02 03:12:41.220356: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-02 03:12:44.216366: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


176/176 - 4s - loss: 1.0266 - accuracy: 0.5187 - val_loss: 0.6860 - val_accuracy: 0.5538 - 4s/epoch - 22ms/step
Epoch 2/50
176/176 - 3s - loss: 0.7111 - accuracy: 0.5448 - val_loss: 0.6679 - val_accuracy: 0.5779 - 3s/epoch - 18ms/step
Epoch 3/50
176/176 - 3s - loss: 0.6943 - accuracy: 0.5627 - val_loss: 0.6635 - val_accuracy: 0.6454 - 3s/epoch - 18ms/step
Epoch 4/50
176/176 - 3s - loss: 0.6816 - accuracy: 0.5755 - val_loss: 0.6533 - val_accuracy: 0.6479 - 3s/epoch - 18ms/step
Epoch 5/50
176/176 - 3s - loss: 0.6723 - accuracy: 0.5913 - val_loss: 0.6487 - val_accuracy: 0.6713 - 3s/epoch - 17ms/step
Epoch 6/50
176/176 - 3s - loss: 0.6592 - accuracy: 0.6094 - val_loss: 0.6450 - val_accuracy: 0.5894 - 3s/epoch - 18ms/step
Epoch 7/50
176/176 - 3s - loss: 0.6530 - accuracy: 0.6170 - val_loss: 0.6237 - val_accuracy: 0.6462 - 3s/epoch - 18ms/step
Epoch 8/50
176/176 - 3s - loss: 0.6429 - accuracy: 0.6276 - val_loss: 0.6154 - val_accuracy: 0.6433 - 3s/epoch - 18ms/step
Epoch 9/50
176/176 - 3s - l